In [14]:
import pandas as pd
import numpy as np

In [15]:
# Устанавливаем формат отображения (округление до 2 знаков после запятой)
pd.options.display.float_format = '{:.2f}'.format
# Чтение из файла auto.json
df = pd.read_json('../data/auto.json', orient='records').set_index('CarNumber')
# Чтение из файла surname.json
surname = pd.read_json('../data/surname.json', orient='values')
surname.columns = surname.iloc[0]
surname = surname[1:]

In [16]:
# Новый массив на основе старого
len_new_df = 200
random_state = 21
new_df = df.sample(n=len_new_df, random_state=random_state)
counter = new_df.count()
concat_row = pd.concat([df, new_df])
concat_row

,Refund,Fines,Make,Model
CarNumber,,,,
Y163O8161RUS,2,3200.00,Ford,Focus
E432XX77RUS,1,6500.00,Toyota,Camry
7184TT36RUS,1,2100.00,Ford,Focus
X582HE161RUS,2,2000.00,Ford,Focus
92918M178RUS,1,5700.00,Ford,Focus
...,...,...,...,...
8182XX154RUS,1,200.00,Ford,Focus
X796TH96RUS,1,500.00,Ford,Focus
T011MY163RUS,2,4000.00,Ford,Focus


In [17]:
# Генерация столбца Year
np.random.seed(21)
year_series = np.random.randint(1980, 2020, size=len(concat_row))
concat_row['Year'] = year_series
fines = concat_row.copy()
fines

,Refund,Fines,Make,Model,Year
CarNumber,,,,,
Y163O8161RUS,2,3200.00,Ford,Focus,1989
E432XX77RUS,1,6500.00,Toyota,Camry,1995
7184TT36RUS,1,2100.00,Ford,Focus,1984
X582HE161RUS,2,2000.00,Ford,Focus,2015
92918M178RUS,1,5700.00,Ford,Focus,2014
...,...,...,...,...,...
8182XX154RUS,1,200.00,Ford,Focus,1981
X796TH96RUS,1,500.00,Ford,Focus,1992
T011MY163RUS,2,4000.00,Ford,Focus,2007


In [18]:
# Создаем новый фрейм из Surname и CarNumber
uniq_number_car = fines.index.unique()
uniq_number_car = uniq_number_car.to_frame().reset_index(drop=True)
value_surname = surname.groupby('NAME').size().sort_values(ascending=False)
value_surname = value_surname.to_frame().reset_index().drop(0, axis=1)
np.random.seed(21)
value_surname = pd.Series(np.random.choice(value_surname['NAME'], size=len(uniq_number_car)))
owners = pd.DataFrame({'CarNumber': uniq_number_car['CarNumber'], 'NAME': value_surname})
owners

,CarNumber,NAME
0,Y163O8161RUS,RICHARDSON
1,E432XX77RUS,ROSS
2,7184TT36RUS,MORGAN
3,X582HE161RUS,BAILEY
4,92918M178RUS,LOPEZ
...,...,...
526,O136HO197RUS,CAMPBELL
527,O22097197RUS,HALL
528,M0309X197RUS,BAKER
529,O673E8197RUS,DIAZ


In [19]:
# Добавление 5 новых данных в фрейм fines
data = {
    'CarNumber': ['А123BC77', 'К456DE77', 'А123BC77', 'М789FG77', 'К456DE77'],
    'Refund': ['2', '1', '2', '1', '1'],
    'Fines': ['200', '100', '5438', '673', '890'],
    'Make': ['Audi', 'Chery', 'Москвич', 'Haval', 'Nissan'],
    'Model': ['A6', 'Tigga 4', '3', 'F7', 'Quashkai'],
    'Year': ['2018', '2017', '2019', '2019', '2018']
}
data_fines = pd.DataFrame(data).set_index('CarNumber')
fines = pd.concat([fines, data_fines])
fines

,Refund,Fines,Make,Model,Year
CarNumber,,,,,
Y163O8161RUS,2,3200.00,Ford,Focus,1989
E432XX77RUS,1,6500.00,Toyota,Camry,1995
7184TT36RUS,1,2100.00,Ford,Focus,1984
X582HE161RUS,2,2000.00,Ford,Focus,2015
92918M178RUS,1,5700.00,Ford,Focus,2014
...,...,...,...,...,...
А123BC77,2,200,Audi,A6,2018
К456DE77,1,100,Chery,Tigga 4,2017
А123BC77,2,5438,Москвич,3,2019


In [20]:
# Удаление последних 20 значений из value_counts
owners_df = owners.iloc[:-20]
data = {
    'CarNumber': ['9878EE757RUS', '1238EE721RUS', '6208EE756RUS'],
    'NAME': ['VOLKOV', 'PETROV', 'IVANOV']
}
data_owners = pd.DataFrame(data)
owners = pd.concat([owners_df, data_owners])
owners

,CarNumber,NAME
0,Y163O8161RUS,RICHARDSON
1,E432XX77RUS,ROSS
2,7184TT36RUS,MORGAN
3,X582HE161RUS,BAILEY
4,92918M178RUS,LOPEZ
...,...,...
509,O50197197RUS,WRIGHT
510,7608EE777RUS,HILL
0,9878EE757RUS,VOLKOV
1,1238EE721RUS,PETROV


In [21]:
# Новый фрейм данных должен содержать только те номера автомобилей, которые существуют в обоих фреймах данных
inner_fines = pd.merge(fines, owners,  on='CarNumber',how='inner')
inner_fines

,CarNumber,Refund,Fines,Make,Model,Year,NAME
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,RICHARDSON
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995,ROSS
2,7184TT36RUS,1,2100.00,Ford,Focus,1984,MORGAN
3,X582HE161RUS,2,2000.00,Ford,Focus,2015,BAILEY
4,92918M178RUS,1,5700.00,Ford,Focus,2014,LOPEZ
...,...,...,...,...,...,...,...
894,8182XX154RUS,1,200.00,Ford,Focus,1981,SMITH
895,X796TH96RUS,1,500.00,Ford,Focus,1992,WATSON
896,T011MY163RUS,2,4000.00,Ford,Focus,2007,SANDERS
897,T341CC96RUS,2,1000.00,Volkswagen,Passat,2005,PEREZ


In [22]:
# новый фрейм данных должен содержать все номера автомобилей, которые существуют в обоих фреймах данных
outer_fines = pd.merge(fines, owners,  on='CarNumber',how='outer')
outer_fines

,CarNumber,Refund,Fines,Make,Model,Year,NAME
0,1238EE721RUS,NaN,NaN,NaN,NaN,NaN,PETROV
1,6208EE756RUS,NaN,NaN,NaN,NaN,NaN,IVANOV
2,704687163RUS,2,1400.00,Ford,Focus,2004,ADAMS
3,704787163RUS,2,2800.00,Ford,Focus,1992,MORGAN
4,704987163RUS,2,8594.59,Ford,Focus,1985,MITCHELL
...,...,...,...,...,...,...,...
928,А123BC77,2,200,Audi,A6,2018,NaN
929,А123BC77,2,5438,Москвич,3,2019,NaN
930,К456DE77,1,100,Chery,Tigga 4,2017,NaN
931,К456DE77,1,890,Nissan,Quashkai,2018,NaN


In [23]:
# новый фрейм данных должен содержать только номера автомобилей из фрейма данных о штрафах
left_fines = pd.merge(fines, owners,  on='CarNumber',how='left')
left_fines

,CarNumber,Refund,Fines,Make,Model,Year,NAME
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,RICHARDSON
1,E432XX77RUS,1,6500.00,Toyota,Camry,1995,ROSS
2,7184TT36RUS,1,2100.00,Ford,Focus,1984,MORGAN
3,X582HE161RUS,2,2000.00,Ford,Focus,2015,BAILEY
4,92918M178RUS,1,5700.00,Ford,Focus,2014,LOPEZ
...,...,...,...,...,...,...,...
925,А123BC77,2,200,Audi,A6,2018,NaN
926,К456DE77,1,100,Chery,Tigga 4,2017,NaN
927,А123BC77,2,5438,Москвич,3,2019,NaN
928,М789FG77,1,673,Haval,F7,2019,NaN


In [24]:
# новый фрейм данных должен содержать только номера автомобилей из фрейма данных владельцев
right_fines = pd.merge(fines, owners,  on='CarNumber',how='right')
right_fines

,CarNumber,Refund,Fines,Make,Model,Year,NAME
0,Y163O8161RUS,2,3200.00,Ford,Focus,1989,RICHARDSON
1,Y163O8161RUS,2,1600.00,Ford,Focus,1980,RICHARDSON
2,E432XX77RUS,1,6500.00,Toyota,Camry,1995,ROSS
3,E432XX77RUS,2,13000.00,Toyota,Camry,2018,ROSS
4,7184TT36RUS,1,2100.00,Ford,Focus,1984,MORGAN
...,...,...,...,...,...,...,...
897,7608EE777RUS,1,4000.00,Skoda,Octavia,2000,HILL
898,7608EE777RUS,1,4000.00,Skoda,Octavia,1991,HILL
899,9878EE757RUS,NaN,NaN,NaN,NaN,NaN,VOLKOV
900,1238EE721RUS,NaN,NaN,NaN,NaN,NaN,PETROV


In [25]:
# Создание сводной таблицы
pivot_table = pd.pivot_table(fines, values='Fines', index=['Make', 'Model'], columns='Year', aggfunc='sum')
pivot_table

Year                         1980      1981      1982     1983     1984  \
Make       Model                                                          
Audi       A6                 NaN       NaN       NaN      NaN      NaN   
           Witout model       NaN       NaN       NaN      NaN      NaN   
BMW        Witout model       NaN       NaN       NaN      NaN      NaN   
Chery      Tigga 4            NaN       NaN       NaN      NaN      NaN   
Ford       Focus        110294.59 408983.76 165883.76 64800.00 96989.17   
           Mondeo             NaN       NaN       NaN      NaN      NaN   
Haval      F7                 NaN       NaN       NaN      NaN      NaN   
Nissan     Quashkai           NaN       NaN       NaN      NaN      NaN   
Skoda      Octavia        2400.00       NaN   7300.00 11594.59      NaN   
Toyota     Camry         12000.00   8594.59       NaN  7200.00      NaN   
           Corolla            NaN       NaN   2000.00      NaN      NaN   
Volkswagen Golf          30900.00       NaN       NaN  8594.59   300.00   
           Jetta              NaN       NaN       NaN      NaN      NaN   
           Passat             NaN   1600.00       NaN  3200.00 10000.00   
           Touareg            NaN       NaN       NaN      NaN      NaN   
           Witout model       NaN       NaN       NaN      NaN      NaN   
Volvo      Witout model       NaN       NaN   6800.00      NaN      NaN   
Москвич    3                  NaN       NaN       NaN      NaN      NaN   

Year                         1985     1986      1987      1988      1989  ...  \
Make       Model                                                          ...   
Audi       A6                 NaN      NaN       NaN       NaN       NaN  ...   
           Witout model       NaN      NaN       NaN       NaN   4200.00  ...   
BMW        Witout model       NaN      NaN       NaN       NaN       NaN  ...   
Chery      Tigga 4            NaN      NaN       NaN       NaN       NaN  ...   
Ford       Focus        162683.76 96589.17 125700.00 111789.17 176094.59  ...   
           Mondeo             NaN      NaN       NaN       NaN   8600.00  ...   
Haval      F7                 NaN      NaN       NaN       NaN       NaN  ...   
Nissan     Quashkai           NaN      NaN       NaN       NaN       NaN  ...   
Skoda      Octavia       10294.59   600.00   5200.00   5200.00  91400.00  ...   
Toyota     Camry              NaN      NaN       NaN       NaN  22400.00  ...   
           Corolla            NaN      NaN  14900.00       NaN   4000.00  ...   
Volkswagen Golf          24000.00      NaN  44800.00       NaN   5800.00  ...   
           Jetta              NaN      NaN       NaN       NaN       NaN  ...   
           Passat         5000.00 15000.00  12300.00       NaN       NaN  ...   
           Touareg        5800.00      NaN       NaN       NaN       NaN  ...   
           Witout model       NaN  7400.00       NaN       NaN       NaN  ...   
Volvo      Witout model       NaN      NaN       NaN       NaN       NaN  ...   
Москвич    3                  NaN      NaN       NaN       NaN       NaN  ...   

Year                         2013      2014      2015     2016      2017  \
Make       Model                                                           
Audi       A6                 NaN       NaN       NaN      NaN       NaN   
           Witout model       NaN       NaN       NaN      NaN       NaN   
BMW        Witout model       NaN       NaN   8594.59      NaN       NaN   
Chery      Tigga 4            NaN       NaN       NaN      NaN       NaN   
Ford       Focus        139394.59 122678.35 209100.00 98089.17 263000.00   
           Mondeo             NaN       NaN       NaN 46200.00       NaN   
Haval      F7                 NaN       NaN       NaN      NaN       NaN   
Nissan     Quashkai           NaN       NaN       NaN      NaN       NaN   
Skoda      Octavia       12594.59    300.00  46394.59   300.00   8594.59   
Toyota     Camry              NaN   1000.0

In [26]:
# Сохранение fines и owners в файлы csv
fines.to_csv('../data/data_df/fines.csv')
owners.to_csv('../data/data_df/owners.csv', index=False)